# WIT Dispatcher

In [1]:
import fiona.transform
import time
import asyncio, aiohttp, xmltodict
import string

if not hasattr(asyncio, 'create_task'):
    asyncio.create_task = asyncio.ensure_future

Polygons can be read directly from the zipped shapefile supplied. (However, an optimisation would be to store the vector data in the target projection.)

In [2]:
def generate_polygons(max_index=None, max_length=None):
    """Produce indexed WGS84 polygons from shapefile"""
    source = "zip://Queensland_dominant_wetland_areas_22042020.zip"
    with fiona.open(source) as collection:
        #collection.ignore_fields = list(collection.schema['properties']) # read fewer columns
        for i, record in enumerate(collection):
            if max_length and record['properties']['Shape_Leng'] > max_length:
                continue
            # Note, reprojection is very slow, and could instead be prepared prior to runtime. 
            geom = fiona.transform.transform_geom(collection.crs, 'EPSG:4326', record['geometry'])
            geom['coordinates'] = [[[lon, lat] for (lat, lon) in ring] for ring in geom['coordinates']]
            yield i, geom
            if max_index and i >= max_index:
                break

Polygons must be encoded into WPS *execute* requests, together with a time interval. For WIT, it is unimportant whether all time is processed in a single interval, or divided into multiple successive increments.

In [3]:
request_template = """
<wps:Execute version="1.0.0" service="WPS" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.opengis.net/wps/1.0.0" xmlns:wfs="http://www.opengis.net/wfs" xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc" xmlns:wcs="http://www.opengis.net/wcs/1.1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsAll.xsd">
    <ows:Identifier>WIT</ows:Identifier>
    <wps:DataInputs>
    <wps:Input>
        <ows:Identifier>geometry</ows:Identifier>
        <wps:Data>
        <wps:ComplexData>{"type":"FeatureCollection","features":[{"type":"Feature","geometry":%s]}</wps:ComplexData>
        </wps:Data>
    </wps:Input>
    <wps:Input>
        <ows:Identifier>start</ows:Identifier>
        <wps:Data>
        <wps:ComplexData>"01-01-2000"</wps:ComplexData>
        </wps:Data>
    </wps:Input>
    <wps:Input>
        <ows:Identifier>end</ows:Identifier>
        <wps:Data>
        <wps:ComplexData>"2001"</wps:ComplexData>
        </wps:Data>
    </wps:Input>
    </wps:DataInputs>
    <wps:ResponseForm>
    <wps:ResponseDocument storeExecuteResponse="true" status="true"/>
    </wps:ResponseForm>
</wps:Execute>
"""
def request(geom):
    return request_template % str(geom).replace("'", '"')

For testing, we just want a few individual requests for the WPS, and to exclude any large polygons. There is also a simpler request example that only targets a pixel drill.

In [4]:
example_requests = ((i, request(poly)) for i, poly in generate_polygons(max_index=10, max_length=2000))

In [5]:
dummy_template = """<wps:Execute version="1.0.0" service="WPS" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns="http://www.opengis.net/wps/1.0.0" xmlns:wfs="http://www.opengis.net/wfs" xmlns:wps="http://www.opengis.net/wps/1.0.0" xmlns:ows="http://www.opengis.net/ows/1.1" xmlns:gml="http://www.opengis.net/gml" xmlns:ogc="http://www.opengis.net/ogc" xmlns:wcs="http://www.opengis.net/wcs/1.1.1" xmlns:xlink="http://www.w3.org/1999/xlink" xsi:schemaLocation="http://www.opengis.net/wps/1.0.0 http://schemas.opengis.net/wps/1.0.0/wpsAll.xsd">
    <ows:Identifier>WOfSDrill</ows:Identifier>
    <wps:DataInputs>
    <wps:Input>
        <ows:Identifier>geometry</ows:Identifier>
        <wps:Data>
        <wps:ComplexData>{"type":"FeatureCollection","features":[{"type":"Feature","geometry":{"type":"Point","coordinates":[146.85029736971987,-32.94459759906837,-1822.7196235501208]}}]}</wps:ComplexData>
        </wps:Data>
    </wps:Input>
    </wps:DataInputs>
    <wps:ResponseForm>
    <wps:ResponseDocument storeExecuteResponse="true" status="true"/>
    </wps:ResponseForm>
</wps:Execute>"""

dummy_requests = ((i, dummy_template) for i in range(4))

Next, to poll the WPS for requests to be performed.

In [6]:
wps_url = 'https://ows.dev.dea.ga.gov.au/wps/?service=WPS&request=Execute'
wps_headers = {'Content-Type': 'text/xml;charset=UTF-8', 'cache-control': 'max-age=0'}
polling_interval = 2.0 # seconds

In [7]:
async def request(session, log, index, request_doc):
    "Execute a single request, through to completion"
    try:
        resp = await session.post(wps_url, data=request_doc, headers=wps_headers)
        assert resp.status == 200
        info = xmltodict.parse(await resp.text())
        url = info['wps:ExecuteResponse']['@statusLocation']
        log[index] = [url]
        assert 'wps:ProcessAccepted' in info['wps:ExecuteResponse']['wps:Status'] 
        while {'wps:ProcessAccepted', 'wps:ProcessStarted'} & set(info['wps:ExecuteResponse']['wps:Status']):
            await asyncio.sleep(polling_interval)
            resp = await session.get(url)
            assert resp.status == 200
            info = xmltodict.parse(await resp.text())
            print('.', end='', flush=True)
        assert 'wps:ProcessSucceeded' in info['wps:ExecuteResponse']['wps:Status']
        log[index].append('Succeeded')
        return info['wps:ExecuteResponse']['wps:ProcessOutputs']['wps:Output']
    except:
        log[index].append('Failed')
        raise

In [8]:
#log = {}
#async with aiohttp.ClientSession() as session: 
#    result = await request(session, log, 1, dummy_template)   
#log

In [9]:
# limit the concurrency

log = {}
results = []
n = 2
sem = asyncio.Semaphore(n)

async with aiohttp.ClientSession() as session:
    
    async def task_wrapper(args):
        try:
            results.append(await request(session, log, *args))
        finally:
            sem.release()
            
    for args in dummy_requests:
        await sem.acquire() 
        asyncio.ensure_future(task_wrapper(args))
        
    # wait for completion before closing session
    for i in range(n):
        await sem.acquire()
        
log

..............................................................................................

{1: ['https://s3.ap-southeast-2.amazonaws.com/dea-dev-wps-results/84c15212-d43e-11eb-80c8-166cbd501bee.xml',
  'Succeeded'],
 0: ['https://s3.ap-southeast-2.amazonaws.com/dea-dev-wps-results/84e05f68-d43e-11eb-80c8-166cbd501bee.xml',
  'Succeeded'],
 2: ['https://s3.ap-southeast-2.amazonaws.com/dea-dev-wps-results/a1e42590-d43e-11eb-80c8-166cbd501bee.xml',
  'Succeeded'],
 3: ['https://s3.ap-southeast-2.amazonaws.com/dea-dev-wps-results/a2030122-d43e-11eb-80c8-166cbd501bee.xml',
  'Succeeded']}